In [35]:
import pandas as pd
import numpy as np
import datetime

#%matplotlib inline

# Leo csv

In [36]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [37]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [38]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

In [39]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [40]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Borro duplicados y agrego edad

In [42]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [46]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [47]:
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

In [48]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(['fechanacimiento'],axis=1)

In [49]:
# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')
postulantes_genero_y_edad.isnull().sum()

idpostulante    0
sexo            0
edad            0
dtype: int64

In [50]:
postulantes_genero_y_edad.sexo.value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

In [51]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [53]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [54]:
postulaciones = postulaciones.drop_duplicates(keep='last')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Separo gente postulada y no postulada

In [55]:
gente_postulada = postulaciones
gente_postulada = gente_postulada.drop_duplicates(subset=['idpostulante'],keep='last')
gente_postulada = gente_postulada.reset_index().drop(['index','fechapostulacion'],axis=1)

In [56]:
gente_postulada.head()

,idaviso,idpostulante
0,1112345547,NM5M
1,1112305358,5awk
2,1112345828,eo2p
3,1112343340,1d2B
4,1112359639,1QLLO


In [57]:
gente_postulada.shape

(348883, 2)

In [58]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [59]:
tupla_no_postulada = generador_de_sample_serie(avisos_detalle['idaviso'],gente_postulada['idpostulante'],100,100)

In [60]:
tupla_no_postulada.shape

(1000000, 2)

In [61]:
tupla_no_postulada.head()

,idaviso,idpostulante
0,1112390041,0zB26eY
1,1112390041,X9Px1OJ
2,1112390041,Yj5MwlD
3,1112390041,4rWQD1E
4,1112390041,ow5Ko3Z


In [64]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [65]:
postulaciones.shape

(6604534, 3)

In [66]:
tupla_no_postulada = tupla_no_postulada.merge(postulaciones,how='left',on=['idaviso','idpostulante'])

In [67]:
tupla_no_postulada.head()

,idaviso,idpostulante,fechapostulacion
0,1112390041,0zB26eY,NaN
1,1112390041,X9Px1OJ,NaN
2,1112390041,Yj5MwlD,NaN
3,1112390041,4rWQD1E,NaN
4,1112390041,ow5Ko3Z,NaN


In [68]:
tupla_no_postulada.shape

(1000000, 3)

In [32]:
tupla_no_postulada = tupla_no_postulada.fillna(0)

In [33]:
tupla_no_postulada = tupla_no_postulada.loc[tupla_no_postulada.fechapostulacion==0]

In [34]:
tupla_no_postulada = tupla_no_postulada.rename(columns={'fechapostulacion': 'y'})

In [35]:
tupla_no_postulada.shape

(999342, 3)

In [36]:
tupla_no_postulada.head()

,idaviso,idpostulante,y
0,1112390041,0zB26eY,0
1,1112390041,X9Px1OJ,0
2,1112390041,Yj5MwlD,0
3,1112390041,4rWQD1E,0
4,1112390041,ow5Ko3Z,0


In [37]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
#gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [38]:
# GENTE POSTULADA
#gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
#                                               isin(postulaciones['idpostulante'].value_counts().index)]
#gente_postulada = gente_postulada.reset_index(drop=True)

In [39]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
#gente_no_postulada = gente_no_postulada.dropna()
#gente_no_postulada = gente_no_postulada.reset_index(drop=True)
#gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [41]:
gente_postulada = gente_postulada.merge(postulantes_genero_y_edad,on='idpostulante',how='left')

In [42]:
gente_postulada.head()

,idaviso,idpostulante,sexo,edad
0,1112345547,NM5M,FEM,48
1,1112305358,5awk,FEM,56
2,1112345828,eo2p,MASC,37
3,1112343340,1d2B,MASC,42
4,1112359639,1QLLO,FEM,40


# Arreglo el df de avisos detalle

In [44]:
avisos_detalle['nombre_area'].unique().shape

(188,)

In [45]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [46]:
print(avisos_detalle['nombre_area'].value_counts())

Ventas                               5256
Administración                       3218
Tecnologia/Sistemas                  3062
Producción                           2268
Atención al Cliente                  1354
Logística                            1307
Oficios                              1180
Recursos Humanos                     1012
Finanzas                              896
Ingenieria                            868
Salud                                 725
Gastronomia                           602
Otros                                 583
Marketing                             579
Secretaria                            459
Construcción                          391
Comercio Exterior                     236
Legales                               235
Diseño y Urbanismo                    128
Minería                               112
Tecnologia/Sistemas de producción     107
Seguros                               106
Gerencia                              102
Comunicacion                      

In [47]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

# Funciones

In [48]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [49]:
#def feature_genero_edad(tupla_idaviso_idpostulante):
#    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
#    return df_return



#def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
#    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
#    return df_return

In [50]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

# Test final

In [51]:
test_final = test_final.merge(postulantes_genero_y_edad,on='idpostulante',how='left')

In [52]:
test_final.head()

,idaviso,idpostulante,sexo,edad
0,739260,6M9ZQR,FEM,42
1,739260,6v1xdL,MASC,31
2,739260,ezRKm9,FEM,36
3,758580,1Q35ej,MASC,69
4,758580,EAN4J6,FEM,32


In [53]:
test_final = feature_area_de_trabajo(test_final)

In [54]:
test_final.head()

,idaviso,idpostulante,sexo,edad,nombre_area
0,739260,6M9ZQR,FEM,42,Ventas
1,739260,6v1xdL,MASC,31,Ventas
2,739260,ezRKm9,FEM,36,Ventas
3,758580,1Q35ej,MASC,69,Tecnologia/Sistemas
4,758580,EAN4J6,FEM,32,Tecnologia/Sistemas


In [55]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [56]:
test_final = test_final.drop(['idaviso','idpostulante'],axis=1)
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [57]:
test_final.head()

,edad,nombre_area,FEM,MASC,NO_DECLARA
0,42,Ventas,1,0,0
1,31,Ventas,0,1,0
2,36,Ventas,1,0,0
3,69,Tecnologia/Sistemas,0,1,0
4,32,Tecnologia/Sistemas,1,0,0


# Tupla postulada y sample no postulado

In [60]:
tupla_postulada = postulaciones.merge(postulantes_genero_y_edad,on='idpostulante',how='left')

In [61]:
tupla_postulada.head()

,idaviso,idpostulante,fechapostulacion,sexo,edad
0,1112257047,NM5M,2018-01-15 16:22:34,FEM,48
1,1111920714,NM5M,2018-02-06 09:04:50,FEM,48
2,1112346945,NM5M,2018-02-22 09:04:47,FEM,48
3,1112345547,NM5M,2018-02-22 09:04:59,FEM,48
4,1112237522,5awk,2018-01-25 18:55:03,FEM,56


In [62]:
tupla_postulada = tupla_postulada.drop('fechapostulacion',axis=1)

In [63]:
tupla_postulada.head()

,idaviso,idpostulante,sexo,edad
0,1112257047,NM5M,FEM,48
1,1111920714,NM5M,FEM,48
2,1112346945,NM5M,FEM,48
3,1112345547,NM5M,FEM,48
4,1112237522,5awk,FEM,56


In [64]:
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [65]:
tupla_postulada.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,48,1,0,0
1,1111920714,NM5M,48,1,0,0
2,1112346945,NM5M,48,1,0,0
3,1112345547,NM5M,48,1,0,0
4,1112237522,5awk,56,1,0,0


In [66]:
tupla_postulada['y'] = 1

In [67]:
tupla_postulada.head()

,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA,y
0,1112257047,NM5M,48,1,0,0,1
1,1111920714,NM5M,48,1,0,0,1
2,1112346945,NM5M,48,1,0,0,1
3,1112345547,NM5M,48,1,0,0,1
4,1112237522,5awk,56,1,0,0,1


In [69]:
tupla_no_postulada = tupla_no_postulada.merge(postulantes_genero_y_edad,on='idpostulante',how='left')

In [70]:
tupla_no_postulada.head()

,idaviso,idpostulante,y,sexo,edad
0,1112390041,0zB26eY,0,MASC,26
1,1112390041,X9Px1OJ,0,MASC,48
2,1112390041,Yj5MwlD,0,MASC,26
3,1112390041,4rWQD1E,0,MASC,25
4,1112390041,ow5Ko3Z,0,FEM,47


In [71]:
tupla_no_postulada = oneHotEncoding(tupla_no_postulada,'sexo')

In [72]:
tupla_no_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,0zB26eY,0,26,0,1,0
1,1112390041,X9Px1OJ,0,48,0,1,0
2,1112390041,Yj5MwlD,0,26,0,1,0
3,1112390041,4rWQD1E,0,25,0,1,0
4,1112390041,ow5Ko3Z,0,47,1,0,0


In [74]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [75]:
tupla_postulada = change_column_order(tupla_postulada, 'y', 0)

In [76]:
tupla_no_postulada = change_column_order(tupla_no_postulada, 'y', 0)

In [77]:
tupla_postulada.head()

,y,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,1,1112257047,NM5M,48,1,0,0
1,1,1111920714,NM5M,48,1,0,0
2,1,1112346945,NM5M,48,1,0,0
3,1,1112345547,NM5M,48,1,0,0
4,1,1112237522,5awk,56,1,0,0


In [78]:
tupla_no_postulada.head()

,y,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,0,1112390041,0zB26eY,26,0,1,0
1,0,1112390041,X9Px1OJ,48,0,1,0
2,0,1112390041,Yj5MwlD,26,0,1,0
3,0,1112390041,4rWQD1E,25,0,1,0
4,0,1112390041,ow5Ko3Z,47,1,0,0


# Borro lo que no necesito

In [79]:
#del avisos_detalle
#del postulantes_genero_y_edad
#del postulaciones

In [80]:
tupla_no_postulada.shape

(999342, 7)

In [81]:
tupla_postulada.shape

(6604534, 7)

# Separo la edad por rango en el sample y el test

In [82]:
tuplas = tupla_no_postulada.append(tupla_postulada.sample(n=999342,random_state=0))

In [83]:
tuplas.head()

,y,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
0,0,1112390041,0zB26eY,26,0,1,0
1,0,1112390041,X9Px1OJ,48,0,1,0
2,0,1112390041,Yj5MwlD,26,0,1,0
3,0,1112390041,4rWQD1E,25,0,1,0
4,0,1112390041,ow5Ko3Z,47,1,0,0


In [84]:
tuplas.tail()

,y,idaviso,idpostulante,edad,FEM,MASC,NO_DECLARA
318335,1,1112279584,KBeqaMQ,22,1,0,0
1780186,1,1112376814,0zPaQQd,36,0,1,0
219711,1,1112270201,eak8m8,32,0,1,0
2491587,1,1112299234,QNa5dxz,29,1,0,0
623022,1,1112257011,NzrpkBD,39,0,1,0


In [85]:
tuplas = tuplas.drop('idpostulante',axis=1)

In [86]:
tuplas['edad'] = pd.cut(tuplas['edad'], range(0, 100, 5))
tuplas['edad'] = tuplas['edad'].astype(str)
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
tuplas['edad'] = tuplas['edad'].map(dic)

In [87]:
test_final['edad'] = pd.cut(test_final['edad'], range(0, 100, 5))
test_final['edad'] = test_final['edad'].astype(str)
test_final['edad'] = test_final['edad'].map(dic)

In [88]:
tuplas['edad'] = tuplas['edad'].fillna(2)

In [89]:
test_final['edad'] = test_final['edad'].fillna(2)

In [90]:
tuplas = feature_area_de_trabajo(tuplas)

In [91]:
tuplas = oneHotEncoding(tuplas,'nombre_area')

In [92]:
tuplas = tuplas.drop(['idaviso'],axis=1)

In [93]:
tuplas.head()

,y,edad,FEM,MASC,NO_DECLARA,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,...,Recursos Humanos,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero
0,0,3.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,7.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,7.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
test_final = oneHotEncoding(test_final,'nombre_area')

In [95]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,...,Recursos Humanos,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero
0,6.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [96]:
tuplas['y'] = tuplas['y'].astype('int32')

# Random Forest

In [97]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [98]:
train_label = []
train_data = []

train_label = tuplas.iloc[:, 0]
train_data = tuplas.iloc[:, tuplas.columns != 'y']

In [99]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [100]:
ModeloRFC.fit(train_data, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [101]:
result = ModeloRFC.predict(test_final)

In [102]:
result.size

100000

In [103]:
import csv

In [104]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [105]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_area_NuevoTrainV3.csv")

In [108]:
prueba = pd.read_csv('../predicciones/RFC_area_NuevoTrainV3.csv')

In [109]:
prueba.sepostulo.value_counts()

1.0    51025
0.0    48975
Name: sepostulo, dtype: int64